In [1]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

In [3]:
cities_df = pd.read_csv("../output_data/city_data.csv")
cities_df.head()

,City Name,Country,Latitude,Longitude,Temperature (F),Humidity,Cloudiness,Wind Speed (MPH),Time of Data Calculation
0,manakara,MG,-22.13,48.02,69.48,87,66,5.26,2020-10-19 23:59:30
1,lucapa,AO,-8.42,20.74,66.42,89,7,2.82,2020-10-20 00:05:17
2,cherskiy,RU,68.75,161.30,20.59,94,7,3.80,2020-10-19 23:58:53
3,hasaki,JP,35.73,140.83,58.30,87,75,9.17,2020-10-19 23:58:49
4,cape town,ZA,-33.93,18.42,56.44,93,20,5.82,2020-10-20 00:03:49


In [4]:
vacation_df = cities_df.loc[(cities_df['Cloudiness'] <= 8) &
              (cities_df['Humidity'] <= 25) &
              (cities_df['Wind Speed (MPH)'] <= 8) &
              (cities_df['Temperature (F)'] <= 85) &
              (cities_df['Temperature (F)'] >= 60) 
              ]
vacation_df

,City Name,Country,Latitude,Longitude,Temperature (F),Humidity,Cloudiness,Wind Speed (MPH),Time of Data Calculation
165,riyadh,SA,24.69,46.72,66.20,25,0,4.70,2020-10-20 00:02:20
199,narnaul,IN,28.04,76.11,72.50,21,0,2.51,2020-10-20 00:05:41
240,hit,IQ,33.64,42.83,71.80,23,0,7.23,2020-10-20 00:05:45
398,aswan,EG,24.09,32.91,77.92,25,0,5.41,2020-10-20 00:06:01
434,kalabo,ZM,-14.97,22.68,70.99,20,0,5.95,2020-10-20 00:06:06
466,nizwa,OM,22.93,57.53,69.48,24,1,3.42,2020-10-20 00:06:09


In [5]:
#Find Nearest Hotel

def nearest_hotel(coord):
    params = {
    'key' : g_key,
    'location' : coord,
    'radius' : 5000,
    'type' : 'lodging',
    'keyword' : 'hotel'
    }

    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    response = requests.get(base_url, params=params)
    data = response.json()

    return data['results'][0]['name']

In [6]:
hotel_list = []

In [7]:
try:
    coord = '24.69,46.72'
    result = nearest_hotel(coord)
    hotel_list.append(result)
    print(f'Adding {result} to list')
    
except:
    print(f'No Hotels with 5,000m of {coord}')

Adding Four Seasons Hotel Riyadh At Kingdom Center to list


In [8]:
try:
    coord = '28.04,76.11'
    result = nearest_hotel(coord)
    hotel_list.append(result)
    print(f'Adding {result} to list')

except:
    print(f'No Hotels with 5,000m of {coord}')

Adding OM HOTEL to list


In [9]:
try:
    coord = '33.64,42.83'
    result = nearest_hotel(coord)
    hotel_list.append(result)
    print(f'Adding {result} to list')
    
except:
    print(f'No Hotels within 5,000m of {coord}')
    hotel_list.append('None')

No Hotels within 5,000m of 33.64,42.83


In [10]:
try:
    coord = '24.09,32.91'
    result = nearest_hotel(coord)
    hotel_list.append(result)
    print(f'Adding {result} to list')

except:
    print(f'No Hotels with 5,000m of {coord}')

Adding Sofitel Legend Old Cataract Aswan to list


In [11]:
try:
    coord = '-14.97,22.68'
    result = nearest_hotel(coord)
    hotel_list.append(result)
    print(f'Adding {result} to list')

except:
    print(f'No Hotels with 5,000m of {coord}')

Adding Sikakubete Guest House Kalabo Town Council to list


In [12]:
try:
    coord = '22.93,57.53'
    result = nearest_hotel(coord)
    hotel_list.append(result)
    print(f'Adding {result} to list')
    
except:
    print(f'No Hotels with 5,000m of {coord}')

Adding Antique Inn - Nizwa to list


In [13]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [14]:
# Create a Heatmap layer
locations = cities_df[["Latitude", "Longitude"]].astype(float)
humidity = cities_df['Humidity'].astype(float)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, point_radius = 3
                                )
heat_layer.opacity = 0.4

In [15]:
# Create a Marker layer
marker_locations = vacation_df[["Latitude", "Longitude"]]
city_list = vacation_df['City Name'].to_list()
country_list = vacation_df['Country'].to_list()

vacation_cities = [
    {'hotel':hotel_list[0], 'city':city_list[0], 'country':country_list[0]},
    {'hotel':hotel_list[1], 'city':city_list[1], 'country':country_list[1]},
    {'hotel':hotel_list[2], 'city':city_list[2], 'country':country_list[2]},
    {'hotel':hotel_list[3], 'city':city_list[3], 'country':country_list[3]},
    {'hotel':hotel_list[4], 'city':city_list[4], 'country':country_list[4]},
    {'hotel':hotel_list[5], 'city':city_list[5], 'country':country_list[5]}
]

info_box_template = """
<dl>
<dt>Hotel</dt><dd>{hotel}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
city_info = [info_box_template.format(**city) for city in vacation_cities]

marker_layer = gmaps.marker_layer(marker_locations, info_box_content=city_info)

In [16]:
#Final Map with heatmap and cities
map_center = (35,0)
fig = gmaps.figure(center=map_center, zoom_level=1.5)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))